In [ ]:
import os
import pandas as pd
import bs4
from urllib.request import urlopen
from bs4 import BeautifulSoup as soup
import re
from tqdm import tqdm
import pdb
import pickle

url_prefix = 'http://vip.stock.finance.sina.com.cn'
url_pattern = r'href=".+target='
title_pattern = r'_blank">.+</a>'
year_pattern = r'20[01][0-9]'

code = '000001'

def getTitle(code):
    try:
        reports ={}
        company_url = f'http://vip.stock.finance.sina.com.cn/corp/go.php/vCB_Bulletin/stockid/{code}/page_type/ndbg.phtml'
        print(f'Processing {code} ')

        page_client = urlopen(company_url)
        page = page_client.read().decode('gbk')
        page_client.close()

        page_soup = soup(page, "html.parser")
        content = page_soup.find('div',{'class':'datelist'})

        all_url_title = content.findAll('a')

        current_id_set = set()
        for v in all_url_title:
            url = url_prefix + re.findall(url_pattern,str(v))[0][6:-9]
            url = url.replace('amp;','')          

            title = re.findall(title_pattern,str(v))[0][8:-4]
            year = re.findall(year_pattern, title)
            if not year: continue
            year = year[0]
            report_id = code+'_'+year
            if report_id not in reports:
                reports[report_id]=url;
                
#         print(reports)
        return reports
    except Exception as e:
        print(e)

getTitle(code)  

def readContent(url):
    try:
        page_client = urlopen(url)
        page = page_client.read().decode('gbk')
        page_client.close()

        report_soup = soup(page, "html.parser", )

        content = str(report_soup.find('pre'))
#         print(content)
        return content
    except Exception as e:
        print(e)

# url = 'http://vip.stock.finance.sina.com.cn/corp/view/vCB_AllBulletinDetail.php?stockid=000004&id=4386458'; 
# readContent(url)

In [ ]:
import os
import pandas as pd
import re
from tqdm import tqdm
df1 = pd.read_excel('out_join2_20190318.xlsx',index_col=0,
            dtype={'Stkcd': str})
df1.head()



In [ ]:

col=df1.iloc[:,0]
#取表中的第3列的所有值
arrs=col.values
#输出结果
ids = list(set(arrs))
import os
import time
counter=0
for i in tqdm(ids):
    reports = getTitle(i)
    retry = 0
    while not reports and retry<3:
        print(f"获取年报地址失败:{i}, retrying..")
        time.sleep(1+1*retry*retry)
        reports = getTitle(i)
        retry += 1

    if not reports:
        print(f"获取年报地址失败:{i}",'*'*50)
        with open('error.txt','a') as f2:
            f2.write(f'{i}\n')
            f2.close()
        continue
    for (f,url) in reports.items():
        file = f"scraping/content/{f}.txt"
        if os.path.exists(file):
            continue
        content = readContent(url)
        retry = 0
        while not content and retry<3:
            print(f"获取年报地址失败:{i}, retrying..")
            time.sleep(0.1+0.5*retry*retry)
            content = readContent(url)
            retry += 1
        if not content:
            print( f"下载文件失败:{url}")
            continue
        
        print(f"下载文件成功:{url}")
        with open(file,'w') as f:
            counter += 1
            f.write(content)
        
        if counter %100 ==0:
            print('*'*80)
            print('下载文件数',counter)
            print('*'*80)

        

        

In [ ]:
df2 = df1[['Stkcd','Reptdt','Stknme']] 
df2
data = df2.to_dict('records')
print(data[0])